In [18]:
#set seed
import numpy as np
np.random.seed(0)

from sklearn.datasets import load_iris
import pandas as pd

# Load the iris dataset
iris = load_iris()

# Create a dataframe from the iris dataset
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Add the target variable to the dataframe
df['target'] = iris.target
df.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

df["sepal_length_bin"] = pd.cut(df["sepal_length"], bins=[0, 5, 6, 7, 10], labels=["0-5", "5-6", "6-7", "7-10"])

# Display the dataframe
df

,sepal_length,sepal_width,petal_length,petal_width,species,sepal_length_bin
0,5.1,3.5,1.4,0.2,0,5-6
1,4.9,3.0,1.4,0.2,0,0-5
2,4.7,3.2,1.3,0.2,0,0-5
3,4.6,3.1,1.5,0.2,0,0-5
4,5.0,3.6,1.4,0.2,0,0-5
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,6-7
146,6.3,2.5,5.0,1.9,2,6-7
147,6.5,3.0,5.2,2.0,2,6-7
148,6.2,3.4,5.4,2.3,2,6-7


In [19]:
df["sepal_length_bin"].value_counts()

sepal_length_bin
5-6     57
6-7     49
0-5     32
7-10    12
Name: count, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
print(train.shape)
print(test.shape)

(120, 6)
(30, 6)


In [34]:
import numpy as np

adf = pd.DataFrame()
adf.index = df["sepal_length_bin"].unique()
adf["train"] = train["sepal_length_bin"].value_counts()
adf["test"] = test["sepal_length_bin"].value_counts()

adf["train_norm"] = adf["train"] / adf["train"].sum()
adf["test_norm"] = adf["test"] / adf["test"].sum()

adf["diff"] = adf["test_norm"] - adf["train_norm"]
adf["ln_ratio"] = (adf["test_norm"].div(adf["train_norm"])).apply(lambda x: np.log(x))

adf["psi"] = adf["diff"] * adf["ln_ratio"]

print(f"{adf["psi"].sum() = }")
adf


adf["psi"].sum() = 0.14364444574310387


,train,test,train_norm,test_norm,diff,ln_ratio,psi
5-6,48,9,0.400000,0.300000,-0.100000,-0.287682,0.028768
0-5,23,9,0.191667,0.300000,0.108333,0.448025,0.048536
6-7,38,11,0.316667,0.366667,0.050000,0.146603,0.007330
7-10,11,1,0.091667,0.033333,-0.058333,-1.011601,0.059010


### test

In [82]:
def sub_psi(e_perc, a_perc):
            """Calculate the actual PSI value from comparing the values.
            Update the actual value to a very small number if equal to zero
            """
            # formula is wrong direction
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return value

actual_percents = adf["train_norm"]
expected_percents = adf["test_norm"]

# actual_percents = adf["test_norm"]
# expected_percents = adf["train_norm"]

tot = 0
for i in range(0, len(expected_percents)):
    res = sub_psi(expected_percents.iloc[i], actual_percents.iloc[i])
    print(f"{res = }")
    tot += res

print(f"{tot = }")


res = 0.028768207245178118
res = 0.04853601160708735
res = 0.007330173709593771
res = 0.05901005318124465
tot = 0.14364444574310387


In [83]:
def sub_psi(e_perc, a_perc):
            """Calculate the actual PSI value from comparing the values.
            Update the actual value to a very small number if equal to zero
            """
            # formula is wrong direction
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return value

actual_percents = adf["test_norm"]
expected_percents = adf["train_norm"]

tot = 0
for i in range(0, len(expected_percents)):
    res = sub_psi(expected_percents.iloc[i], actual_percents.iloc[i])
    print(f"{res = }")
    tot += res

print(f"{tot = }")


res = 0.02876820724517811
res = 0.04853601160708735
res = 0.007330173709593768
res = 0.05901005318124465
tot = 0.14364444574310387
